In [1]:
import os
#from data_loader import load_word_dataset
import random
import numpy as np
import sys
import nltk
from sklearn.model_selection import train_test_split
import string
# Importing libraries
import pandas as pd
import pprint, time


import sklearn_crfsuite
from sklearn_crfsuite import metrics

from nltk.corpus import treebank
from nltk.corpus import brown

from nltk.classify import MaxentClassifier
from sklearn.metrics import f1_score
import pycrfsuite
from nltk.tag import hmm
# from nltk.classify import megam
from sklearn.model_selection import GridSearchCV
from nltk.tag import BrillTaggerTrainer
from nltk.tag import UnigramTagger
from nltk.tag import DefaultTagger
from nltk.metrics import ConfusionMatrix
from sklearn.model_selection import KFold

In [3]:
#download the treebank corpus from nltk

nltk.download('treebank')
  
# reading the Treebank tagged sentences
tagged_sentences = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

[nltk_data] Error loading treebank: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [19]:
from collections import Counter
# Flatten the list of tagged words
tagged_words = [word for sentence in tagged_sentences for word in sentence]

# Count the occurrences of each POS tag
pos_counts = Counter(tag for word,tag in tagged_words)

# Get the total number of words
total_words = len(tagged_words)

# Calculate the proportion of nouns
noun_proportion = pos_counts['NOUN'] / total_words

print(f"Proportion of nouns: {noun_proportion:.4f}")

Proportion of nouns: 0.2867


Brill

In [5]:


def train_and_evaluate_brill(data, num_repetitions, train_prop):

    
    f1_scores = []

    for _ in range(num_repetitions):
        train_set, test_set = train_test_split(data, train_size=train_prop)

        tag1 = DefaultTagger('X')
        unigram_tagger = UnigramTagger(train_set, backoff=tag1)

        templates = nltk.brill.nltkdemo18()
        trainer = BrillTaggerTrainer(templates=templates, initial_tagger=unigram_tagger)

        # Train the Brill Tagger using the templates
        brill_tagger = trainer.train(train_set, max_rules=200)

        # get predictions
        test_untagged_words = [tup[0] for sent in test_set for tup in sent]
        tags = brill_tagger.tag(test_untagged_words)
        brill_preds = [tag for  _,tag in tags]

        test_true_tags = [tup[1] for sent in test_set for tup in sent]
        #f1 = f1_score(brill_preds, test_true_tags, average='weighted')
        
        

        f1 = f1_score(test_true_tags, brill_preds, average='weighted')
        f1_scores.append(f1)

    return f1_scores


Unigram

In [6]:
def train_and_evaluate_unigram(data, num_repetitions, train_prop):

    
    f1_scores = []

    for _ in range(num_repetitions):
        train_set, test_set = train_test_split(data, train_size=train_prop)

        tag1 = DefaultTagger('X')
        unigram_tagger = UnigramTagger(train_set, backoff=tag1)

     
        # get predictions
        test_untagged_words = [tup[0] for sent in test_set for tup in sent]
        unigram_tags = unigram_tagger.tag(test_untagged_words)
        unigram_preds = [tag for  _,tag in unigram_tags]

        test_true_tags = [tup[1] for sent in test_set for tup in sent]

        f1 = f1_score(test_true_tags, unigram_preds, average='weighted')
        f1_scores.append(f1)

    return f1_scores

HMM

In [7]:
def train_and_evaluate_hmm(data, num_repetitions, train_prop):

    
    f1_scores = []

    for _ in range(num_repetitions):
        train_set, test_set = train_test_split(data, train_size=train_prop)
        tagger = nltk.HiddenMarkovModelTagger.train(train_set)
        
        predicted_labels = []
        correct_labels = []

        for i, sent in enumerate(test_set):
            predicted_labels += [tag for _, tag in tagger.tag([word for word, _ in sent])]
            correct_labels += [tag for _, tag in sent]

        f1 = f1_score(correct_labels, predicted_labels, average='weighted')
        f1_scores.append(f1)

    return f1_scores

Gathering results

In [8]:
# random.seed(1234)


# unigram_80 = train_and_evaluate_unigram(data=tagged_sentences, num_repetitions=10, train_prop=0.8)
# brill_80 = train_and_evaluate_brill(data=tagged_sentences, num_repetitions=10, train_prop=0.8)
# hmm_80 = train_and_evaluate_hmm(data=tagged_sentences, num_repetitions=10, train_prop=0.8)

In [9]:
# f1_scores = {
#     "hmm_80": hmm_80,
#     "unigram_80": unigram_80,
#     "brill_80": brill_80
# }



# with open('insert file path here', "w") as file:
#     for model_threshold, f1_score in f1_scores.items():
#         file.write(f"{model_threshold}: {f1_score}/n")